In [3]:
import os 
from docx import Document
import json 

def extract_text_from_docx(docx_file,start_paragraph, end_paragraph):

    try:
        doc = Document(docx_file)
        paragraphs = doc.paragraphs[start_paragraph-1:end_paragraph]
        return '\n'.join(paragraph.text for paragraph in paragraphs) 
    except KeyError as e:
        print(f"Error processing file {docx_file}: {e}")
    #Load .docx file 
    # doc = Document(docx_file) 
    # extracted_text = [] 
        
    
    # start_index =  start_paragraph - 1 
    # end_index = end_paragraph
    # try:
    #     for i,para in enumerate(doc.paragraphs):
    #         if start_index <= i < end_index:
    #             # print(para.text)
    #             extracted_text.append(para.text)
    #             # print(extracted_text)

    #     return '\n'.join(extracted_text)
    # except KeyError as e:
    #     print(f"Error processing file {docx_file}: {e}")
    #     return None


#sub-folder
def list_files(dir,start_para,end_para,name):
    name_text = ['แบบตรวจติดตาม','ภาพประกอบ','โรงเรียน','รายงานผลตรวจ','F-15-02',
                 'One Page','คู่มือตรวจสอบครอบครอง']
    all_files=[] 
    all_extracted_texts = []
    for root,dirs,files in os.walk(dir):
        for file in files:

            file_path = os.path.join(root,file)
            all_files.append(file_path)
    doc_files = [f for f in all_files if f.endswith('.docx')]

    for file_path in doc_files:
        file_name = os.path.basename(file_path)
        if  any(substring in file_name for substring in name_text): 
            continue 

        
        extracted_text = extract_text_from_docx(file_path,start_para,end_para)

        if extracted_text is None or not extracted_text.strip():
            continue



        all_extracted_texts.append({
            'instruction':file_name,
            "input":'',
            "response":extracted_text
        }) 

    with open(name,'w',encoding='utf-8') as json_file:
        json.dump(all_extracted_texts,json_file,ensure_ascii=False,indent=4)
        print(f"data size have :{len(all_extracted_texts)}")
    return all_extracted_texts


# folder_path ='doc_data/ปี 66'
# name = "extreacted_data_nbtc_team.json"
# files = list_files(folder_path,7,50,name)


## Clean text :)

In [2]:
import pandas as pd 
import glob

path = "./output/*.json"
all_files = glob.glob(path)

In [3]:
df = pd.concat((pd.read_json(f) for f in all_files),ignore_index=True)

In [4]:
df.head()

,instruction,input,response
0,ขออนุมัติเดินทาง - ตรวจสำรองจำหน่าย บุรีรั.docx,,\t1.1\tตามหนังสือ สำนักงาน กสทช. เขต ๒๓ ที่ สท...
1,ปะหน้า รายงานการถวายความปลอดภัย - เรียน อภ.docx,,เรื่อง\tรายงานผลการตรวจสอบคลื่นความถี่ 152.750...
2,เบิกค่าใช้จ่าย - กีฬาสี 2564.docx,,\tในการนี้ การเดินทางปฏิบัติภารกิจดังกล่าว ได้...
3,ตัวอย่างขออนุมัติเดินทาง.docx,,ภาค 2 และสำนักงาน กสทช. เขตในสังกัด ...
4,หนังสือขอยายเวลาจากสถานี.docx,,\n ด้วย...


In [5]:
df['instruction'] = df['instruction'].str.replace(r'\.docx','',regex=True)

In [6]:
import re 
#clean instruction
def clean_instruction(text):
    text = re.sub(r'[A-Za-z]','',text)
    # text = text.strip()
    text = re.sub(r'\d+','',text)
    text = re.sub(r'-+','',text)
    text = re.sub(r'\s+',' ',text)

    return text


def clean_response(text):
    pass

In [7]:
df['instruction'] = df['instruction'].apply(clean_instruction)


In [13]:
text_to_filter = ['รายงานการตรวจวัดคลื่นแม่เหล็กไฟฟ้า','แบบฟอร์มใช้รถ','นายสิริวัฒน์ สิริวัฒนานุวัฒน์','บริเวณพื้นที่ประสบอุทกภัย'
                  ,'สรุป ธันยพัฒน์','แบบรายงานผล','การตั้งค่าการใช้งาน','คำอธิบายสไลด์']
pattern = '|'.join(text_to_filter)
df_filtered = df[~df['instruction'].str.contains(pattern,regex=True,na=False)]





In [14]:
df_filtered = df.drop_duplicates(subset='instruction',keep='first')

In [15]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 328 entries, 0 to 399
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  328 non-null    object
 1   input        328 non-null    object
 2   response     328 non-null    object
dtypes: object(3)
memory usage: 10.2+ KB


In [16]:
df_filtered.head(50)

,instruction,input,response
0,ขออนุมัติเดินทาง ตรวจสำรองจำหน่าย บุรีรั,,\t1.1\tตามหนังสือ สำนักงาน กสทช. เขต ๒๓ ที่ สท...
1,ปะหน้า รายงานการถวายความปลอดภัย เรียน อภ,,เรื่อง\tรายงานผลการตรวจสอบคลื่นความถี่ 152.750...
2,เบิกค่าใช้จ่าย กีฬาสี,,\tในการนี้ การเดินทางปฏิบัติภารกิจดังกล่าว ได้...
3,ตัวอย่างขออนุมัติเดินทาง,,ภาค 2 และสำนักงาน กสทช. เขตในสังกัด ...
4,หนังสือขอยายเวลาจากสถานี,,\n ด้วย...
5,รายงานผล ตามมติ ยกเลิกกิจการ รื้อถอน,,\tตามหนังสือ สำนักการอนุญาตประกอบกิจการกระจายเ...
6,นายสิริวัฒน์ สิริวัฒนานุวัฒน์,,2.นายทรงศักดิ์ กิตติวราพล \t\tตำแหน่...
7,อนุมัติเดินทาง ตรวจแผน ส.ค.,,๑.\tเรื่องเดิม\n\t\t๑.๑\tตามหนังสือ สำนักงาน ก...
8,ปะหน้า รายงานผลแจ้งยืนยัน เอฟเอ็ม กำลังส่,,เรียน\tอภภ. 2\n๑.\tเรื่องเดิม\n\t\tตามหนังสือ ...
9,บริเวณพื้นที่ประสบอุทกภัย ตำบลในเมือง อำเ,,3.นายทศคุณ ยิ่งวัฒนาถาวร นตป. ก3\n4.นายอำน...


In [21]:
df_filtered = df_filtered.drop(index=6)



In [22]:
df_filtered = df_filtered.reset_index(drop=True)

,instruction,input,response
0,ขออนุมัติเดินทาง ตรวจสำรองจำหน่าย บุรีรั,,\t1.1\tตามหนังสือ สำนักงาน กสทช. เขต ๒๓ ที่ สท...
1,ปะหน้า รายงานการถวายความปลอดภัย เรียน อภ,,เรื่อง\tรายงานผลการตรวจสอบคลื่นความถี่ 152.750...
2,เบิกค่าใช้จ่าย กีฬาสี,,\tในการนี้ การเดินทางปฏิบัติภารกิจดังกล่าว ได้...
3,ตัวอย่างขออนุมัติเดินทาง,,ภาค 2 และสำนักงาน กสทช. เขตในสังกัด ...
4,หนังสือขอยายเวลาจากสถานี,,\n ด้วย...
5,รายงานผล ตามมติ ยกเลิกกิจการ รื้อถอน,,\tตามหนังสือ สำนักการอนุญาตประกอบกิจการกระจายเ...
6,อนุมัติเดินทาง ตรวจแผน ส.ค.,,๑.\tเรื่องเดิม\n\t\t๑.๑\tตามหนังสือ สำนักงาน ก...
7,ปะหน้า รายงานผลแจ้งยืนยัน เอฟเอ็ม กำลังส่,,เรียน\tอภภ. 2\n๑.\tเรื่องเดิม\n\t\tตามหนังสือ ...
8,บริเวณพื้นที่ประสบอุทกภัย ตำบลในเมือง อำเ,,3.นายทศคุณ ยิ่งวัฒนาถาวร นตป. ก3\n4.นายอำน...
9,บริเวณพื้นที่ประสบอุทกภัย (หนองนาแซง) จังห,,3.นายทศคุณ ยิ่งวัฒนาถาวร นตป. ก3\n4.นายอำน...


In [111]:
# df_filtered.to_json('data_0.1.json',force_ascii=False,orient='records',indent=4)